In [1]:
# Base dependancies:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
# Make an empty DefaultDict
# fill dictionary {year: dataframe}
# build hierarchal dataframe using dictionary

# Example of data location
# https://raw.githubusercontent.com/SethDKelly/NiceRideMN/master/Nice_Ride_data/2010/NiceRide_station_2010.csv

station_dict = defaultdict()
ridership_dict = defaultdict()

for year in [2010 + x for x in range(8)] :
    station_dict[year] = pd.read_csv("https://raw.githubusercontent.com/SethDKelly/NiceRideMN/master/Nice_Ride_data/" \
                             +str(year)+"/NiceRide_station_"+str(year)+".csv")

    ridership_dict[year] = pd.read_csv("https://raw.githubusercontent.com/SethDKelly/NiceRideMN/master/Nice_Ride_data/" \
                             +str(year)+"/NiceRide_trip_history_"+str(year)+".csv")

NR_station = pd.concat(station_dict)
NR_ridership = pd.concat(ridership_dict)

In [3]:
NR_station.head()

Terminal                           Station   Latitude  Longitude  \
2010 0     30000                100 Main Street SE  44.984892 -93.256551   
     1     30001       25th Street & 33rd. Ave. S.  44.957340 -93.223740   
     2     30002                  Augsburg College  44.966220 -93.238400   
     3     30003  Plymouth Ave N. &  N. Oliver Ave  44.991412 -93.306569   
     4     30004            11th Street & Hennepin  44.975340 -93.278690   

        Ndocks  
2010 0      23  
     1      15  
     2      21  
     3      23  
     4      23

In [4]:
# for model building
from sklearn.cluster import KMeans

# for optimizations
from sklearn.decomposition import PCA

In [7]:
NR_station['Cluster'] = int(0)

for year in [2010 + x for x in range(8)] :
    
    distance_matrix = pd.DataFrame() # Reinstantiate the matrix DataFrame so size row-col reset
    
    # Read in the distance matrix that has been already computed
    distance_matrix = pd.read_csv("https://raw.githubusercontent.com/SethDKelly/NiceRideMN/master/Nice_Ride_data/" \
                               +str(year)+"/distance_matrix_"+str(year)+".csv",index_col=0)
    # Optimize distance matrix to two primary x-y components
    
    pca = PCA(n_components=2).fit_transform(distance_matrix)
    NR_station.loc[year, 'Cluster'] =  KMeans(n_clusters=6, n_init=200).fit(pca).labels_

In [8]:
NR_station

Terminal                                Station   Latitude  \
2010 0       30000                     100 Main Street SE  44.984892   
     1       30001            25th Street & 33rd. Ave. S.  44.957340   
     2       30002                       Augsburg College  44.966220   
     3       30003       Plymouth Ave N. &  N. Oliver Ave  44.991412   
     4       30004                 11th Street & Hennepin  44.975340   
     5       30005               Hennepin & 5th Street NE  44.988890   
     6       30006                          YWCA Downtown  44.972217   
     7       30007                  9th Street & 4th Ave.  44.972650   
     8       30008                       Midtown Exchange  44.949130   
     9       30009              4th Street & 13th Ave. SE  44.981020   
     10      30010                 Hennepin & Laurel Ave.  44.973910   
     11      30011          North 2nd Street & 3rd Ave. N  44.986078   
     12      30012                Mpls. Convention Center  44.970098   
     13      30013                  Lake Street & Lyndale  44.949640   
     14      30014                  Franklin & 21st. Ave.  44.962670   
     15      30015                     Washington & Cedar  44.973440   
     16      30016                      Walker Art Center  44.967460   
     17      30017                  22nd Street & Lyndale  44.960744   
     18      30018                                   MCAD  44.957280   
     19      30019                            Blegen Hall  44.972169   
     20      30020                            YWCA Uptown  44.951310   
     21      30021                Washington & 2nd Ave. N  44.983830   
     22      30022                  26th Street & Lyndale  44.955480   
     23      30023                  Chicago & 27th Street  44.953396   
     24      30024                          Kolthoff Hall  44.973799   
     25      30025                        Williamson Hall  44.977406   
     26      30026                              Lind Hall  44.974190   
     27      30027                         Wilson Library  44.970653   
     28      30028                 Washington & Marquette  44.981120   
     29      30029                5th Street & 3rd Ave. S  44.977390   
...            ...                                    ...        ...   
2017 171     30174           Hiawatha Ave & E 50th Street  44.912943   
     172     30175         18th Ave Trail & 2nd Street NE  45.006321   
     173     30176    Washington Ave N & Cedar Lake Trail  44.985406   
     174     30177           3rd Street NE & Lowry Ave NE  45.013040   
     175     30178             5th Street NE & 1st Ave NE  44.990328   
     176     30179                Park Ave & 4th Street S  44.975895   
     177     30180               3rd Street N & 8th Ave N  44.987996   
     178     30181               5th Ave N & 4th Street N  44.984460   
     179     30182                           Sanford Hall  44.980831   
     180     30183                     Weisman Art Museum  44.973607   
     181     30184        Hennepin Ave & S Washington Ave  44.981895   
     182     30185             5th Street NE & 3rd Ave NE  44.991821   
     183     30186           S 5th Street & Nicollet Mall  44.978665   
     184     30187          Groveland Ave & Clifton Place  44.966145   
     185     30188               S 8th Street & 2nd Ave S  44.974795   
     186     30189       Delaware Street SE & 26th Ave SE  44.972128   
     187     30190            7th Street SE & 10th Ave SE  44.985576   
     188     30191  University Ave SE & Bedford Street SE  44.968760   
     189     30192                             Cooke Hall  44.976303   
     190     30193                         Mariucci Arena  44.977899   
     191     30194        23rd Ave SE & University Ave SE  44.974397   
     192     30195                            Willey Hall  44.972404   
     193     30196                      Civil Engineering  44.976096   
     194     30197            15th Ave SE & 4th St

In [9]:
# Perform necessary imports for Bokeh plotting
from bokeh.io import output_file, show
from bokeh.plotting import figure

# Import the models modules
from bokeh.models import ColumnDataSource, HoverTool, Slider

# Import the layout modules
from bokeh.layouts import widgetbox, row

# To build graph as an application
from bokeh.io import curdoc

# For cluster coloring
from bokeh.models import ColorMapper
from bokeh.palettes import Spectral6

In [15]:
sorted(NR_station.loc[2017,:].Cluster.unique())
#NR_station.loc[:,:].Longitude.values.max()

#                            ('ID','@terminal')
#                            ('Name', '@station')
#                            

[0, 1, 2, 3, 4, 5]

In [20]:
# Make the ColumnDataSource: source
source = ColumnDataSource(data={
    'x'        : NR_station.loc[2017,:].Longitude,
    'y'        : NR_station.loc[2017,:].Latitude,
    'terminal' : NR_station.loc[2017,:].Terminal,
    'station'  : NR_station.loc[2017,:].Station,
    'cluster'  : NR_station.loc[2017,:].Cluster
    })


# Setting boundaries for x-y axis ranges
xmin, xmax = (NR_station.loc[:,:].Longitude.values.min() - .01) , (NR_station.loc[:,:].Longitude.values.max() + .01)
ymin, ymax = (NR_station.loc[:,:].Latitude.values.min() - .01) , (NR_station.loc[:,:].Latitude.values.max() + .01)


# Create the figure: plot
plot = figure(plot_height=600, plot_width=900,
              x_range = (xmin, xmax), y_range = (ymin, ymax))

# Creating hovertool tip
hover = HoverTool(tooltips=[('Terminal', '@terminal'), 
                           ('Name', '@station'),
                           ('Location', '($x, $y)'),
                           ('Cluster', '@cluster')])

# Set the legend.location attribute of the plot to 'top_right'
plot.legend.location = 'top_right'

# adding hovertool to plot
plot.add_tools(hover)


# Add the color mapper to the circle glyph
plot.circle(x='x', y='y', source=source,
            fill_alpha=0.8, legend='cluster'
           )

# Output the file and show the figure
output_file('niceride.html')
show(plot)

